### Using Autoloader to incrementally load the data

Store the path of the containers in the `variables`

In [0]:
storage_account_name = "dlecommerceproject"
raw_container_name="raw"
bronze_container_name="bronze"
raw_path = f"abfss://{raw_container_name}@{storage_account_name}.dfs.core.windows.net"
bronze_path = f"abfss://{bronze_container_name}@{storage_account_name}.dfs.core.windows.net"
checkpoint_path = f"abfss://{bronze_container_name}@{storage_account_name}.dfs.core.windows.net/checkpoint"


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ecommerce_catalog.ecommerce_schema;

Read the Customer data from raw container 

In [0]:

raw_df = spark.readStream\
  .format("cloudFiles")\
  .option("cloudFiles.format", "csv")\
  .option("cloudFiles.schemaLocation", checkpoint_path)\
  .load(raw_path)


Write the Customer data to the bronze container

In [0]:
raw_df.writeStream\
  .option("checkpointLocation", checkpoint_path)\
  .trigger(processingTime='10 seconds')\
  .start(f"{bronze_path}/Customers")